In [9]:
import pandas as pd
import pathlib
import os
import json
import itertools
import numpy as np
import gzip

root = pathlib.Path("/data/rbg/shared/datasets/TCGA/")

In [2]:
signature_gene_mapping = pd.read_excel("copynum_signatures.xlsx").drop(0).rename(columns={"Supplementary Data 2. Annotation of copy number segments": "Signature"}).set_index("Signature")

In [19]:
signature_gene_mapping.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 1043,Unnamed: 1044,Unnamed: 1045,Unnamed: 1046,Unnamed: 1047,Unnamed: 1048,Unnamed: 1049,Unnamed: 1050,Unnamed: 1051,Unnamed: 1052
Signature,,,,,,,,,,,,,,,,,,,,,
chr2:1-15244284.BeroukhimS2.2p25.3.del.,26751,52.0,285016.0,129787.0,391343.0,54221.0,7173.0,7837.0,23040.0,730811.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.p wholearm.chr2:1-93300000,26751,52.0,285016.0,129787.0,391343.0,54221.0,7173.0,7837.0,23040.0,730811.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chr15:56490060-57176541.Basal.15q22.1-107.del,3990,102.0,664618.0,54629.0,79811.0,54778.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chr22:23139372-23249329.BeroukhimS5..amp.,23384,135.0,646023.0,51733.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chr12:97551177-99047626.BeroukhimS2.12q23.1.del.ANKS1B,121457,317.0,56899.0,196472.0,23074.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
print(mygene.get_fields("species"))

AttributeError: module 'mygene' has no attribute 'get_fields'

In [58]:

import mygene
import typing
import functools
import shelve

# ""assume hg38"" POS



def wrap_class_method_with_disk_cache(cls, method_name, cache_file):
    orig = getattr(cls, method_name)

    @functools.wraps(orig)
    def wrapper(self, *args, **kwargs):
        key = (args, tuple(sorted(kwargs.items())))
        key_str = str(key)
        with shelve.open(cache_file) as db:
            if key_str in db:
                return db[key_str]
            result = orig(self, *args, **kwargs)
            db[key_str] = result
            return result

    setattr(cls, method_name, wrapper)


wrap_class_method_with_disk_cache(mygene.MyGeneInfo, 'querymany', 'querymany_tmp.db')
mg = mygene.MyGeneInfo()

def get_ch_pos(entrez_ids):
    results = mg.querymany(entrez_ids, scopes='entrezgene', fields='genomic_pos', species='human')

    for res in results:
        gene_id = res.get('query')
        pos = res.get('genomic_pos')
        if pos:
            if isinstance(pos, list):  # sometimes multiple isoforms
                # typing.assert_never("list?!?")
                for p in pos:
                    yield (f"chr{p['chr']}", p['start'], p['end'])
            else:
                yield (f"chr{pos['chr']}", pos['start'], pos['end'])
        else:
            print(f"Entrez ID {gene_id}: No hg38 coordinates found")

def get_ch_pos_dd(entrez_ids):
    results = mg.querymany(entrez_ids, scopes='entrezgene', fields='genomic_pos', species='human')

    for res in results:
        gene_id = res.get('query')
        pos = res.get('genomic_pos')
        if pos:
            if isinstance(pos, list):  # sometimes multiple isoforms
                # typing.assert_never("list?!?")
                yield (f"chr{pos[0]['chr']}", pos[0]['start'], pos[0]['end'])
            else:
                yield (f"chr{pos['chr']}", pos['start'], pos['end'])
        else:
            print(f"Entrez ID {gene_id}: No hg38 coordinates found")

for idx, row in list(signature_gene_mapping.iterrows())[0:1]:
    entrez_ids = row.dropna().astype(int).to_list()  # ensure ints
    ch_poss = list(get_ch_pos_dd(entrez_ids))
    ch_pos_df = pd.DataFrame(ch_poss, columns=["chromosome", "start", "end"])
    ch_pos_df["chromosome"] = ch_pos_df["chromosome"].apply(lambda x: x.split("_")[0].replace("HSCHR", ""))

    print(ch_pos_df)
    
    





Entrez ID 391343: No hg38 coordinates found
Entrez ID 150622: No hg38 coordinates found
Entrez ID 400940: No hg38 coordinates found
Entrez ID 339788: No hg38 coordinates found
Entrez ID 348738: No hg38 coordinates found
   chromosome     start       end
0        chr2    217730    266398
1        chr2    264140    278283
2        chr2    279558    288851
3        chr2    663877    677406
4        chr2      1089     50794
5        chr2   1374066   1543711
6        chr2   1631887   1744852
7        chr2   1789113   2331664
8        chr2   2319232   2327110
9        chr2   3188925   3377830
10       chr2      9880     94564
11       chr2    106831    110463
12       chr2   3541430   3558333
13       chr2   3575260   3580920
14       chr2   3594832   3644644
15       chr2   3658200   3702671
16       chr2   5692384   5701385
17       chr2   6840570   6866635
18       chr2   6865557   6898239
19       chr2   6917412   7068286
20       chr2   8678845   8684461
21       chr2   8721081   883763